In [116]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.linear_model import Lasso

from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt
 
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))
sns.set()


In [117]:
train = pd.read_csv('~/python/kaggle/housing_prices/train.csv')
test = pd.read_csv('~/python/kaggle/housing_prices/test.csv')
# print(train.describe())
train['SalePrice'] = np.log1p(train['SalePrice'])

In [118]:
joint = pd.concat([train, test],axis=0)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [119]:
null_cols = joint.columns[joint.isnull().any()]
quantity_null_col = joint.isnull().any().sum()
quantity_per_col  = joint[null_cols].isnull().sum()

print(quantity_per_col)
print('')
print("number of features with null columns")
print(quantity_null_col)

Alley           2721
BsmtCond          82
BsmtExposure      82
BsmtFinSF1         1
BsmtFinSF2         1
BsmtFinType1      79
BsmtFinType2      80
BsmtFullBath       2
BsmtHalfBath       2
BsmtQual          81
BsmtUnfSF          1
Electrical         1
Exterior1st        1
Exterior2nd        1
Fence           2348
FireplaceQu     1420
Functional         2
GarageArea         1
GarageCars         1
GarageCond       159
GarageFinish     159
GarageQual       159
GarageType       157
GarageYrBlt      159
KitchenQual        1
LotFrontage      486
MSZoning           4
MasVnrArea        23
MasVnrType        24
MiscFeature     2814
PoolQC          2909
SalePrice       1459
SaleType           1
TotalBsmtSF        1
Utilities          2
dtype: int64

number of features with null columns
35


In [120]:
#---------------
#LotFrontage Imputation
imputer = SimpleImputer()
imputer_mode = SimpleImputer(strategy='most_frequent')

#imputer = SimpleImputer(Strategy = 'median')
imputed_data = imputer.fit_transform(joint['LotFrontage'].values.reshape(2919, 1))
joint['LotFrontage'] = imputed_data

#Alley
# joint['Alley'] = joint['Alley'].replace({'NA':1, 'Pave':2, 'Grvl':3})
joint['Alley'].describe()
joint['Alley'] = joint['Alley'].fillna(1)
joint['Alley'] = joint['Alley'].replace({'Grvl':2, 'Pave':3})

#MasVnrType
# lb = LabelBinarizer()
imputed_data = imputer_mode.fit_transform(joint['MasVnrType'].values.reshape(2919,1))
joint['MasVnrType'] = imputed_data

#-------Street-----------
joint['Street'] = joint['Street'].replace({'Grvl':1, 'Pave':0})

#---------LotShape---------
joint['LotShape'] = joint['LotShape'].replace({'Reg':4, 'IR1':3, 'IR2':2, 'IR3':1})

#------LandContour
joint['LandContour'] = joint['LandContour'].replace({'Lvl':4, 'Bnk':3, 'HLS':2, 'Low':1})

#----------Utilities---------
joint['Utilities'] = joint['Utilities'].fillna('AllPub')
joint['Utilities'] = joint['Utilities'].replace({'AllPub':4, 'NoSewr':3, 'NoSeWa':2, 'ELO':1})

#---------LandSlope
joint['LandSlope'] = joint['LandSlope'].replace({'Gtl':3, 'Mod':2, 'Sev':1})

#------MasVnrArea---
joint['MasVnrArea'] = imputer.fit_transform(joint['MasVnrArea'].values.reshape(2919,1))

#------------Exterqual-----------
joint['ExterQual'] = joint['ExterQual'].replace({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1})

#------ExternCond-------
joint['ExterCond'] = joint['ExterCond'].replace({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1})

#-----BsmtQual------
joint['BsmtQual'] = joint['BsmtQual'].fillna(1)
joint['BsmtQual'] = joint['BsmtQual'].replace({'Ex':6, 'Gd':5, 'TA':4, 'Fa':3, 'Po':2})

#-----BsmtCond
joint['BsmtCond'] = joint['BsmtCond'].fillna(1)
joint['BsmtCond'] = joint['BsmtCond'].replace({'Ex':6, 'Gd':5, 'TA':4, 'Fa':3, 'Po':2})

#--------BsmtExposure
joint['BsmtExposure'] = joint['BsmtExposure'].fillna(1)
joint['BsmtExposure'] = joint['BsmtExposure'].replace({ 'Gd':5, 'Av':4, 'Mn':3, 'No':2})

#BsmtFinType1
joint['BsmtFinType1'] = joint['BsmtFinType1'].fillna(1)
joint['BsmtFinType1'] = joint['BsmtFinType1'].replace({'Unf':2, 'LwQ':3, 'Rec':4, 'BLQ':5, 'ALQ':6, 'GLQ':7})

#BsmtFinType2
joint['BsmtFinType2'] = joint['BsmtFinType2'].fillna(1)
joint['BsmtFinType2'] = joint['BsmtFinType2'].replace({'Unf':2, 'LwQ':3, 'Rec':4, 'BLQ':5, 'ALQ':6, 'GLQ':7})

#Heating
joint['HeatingQC'] = joint['HeatingQC'].replace({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})

#CentralAir
joint['CentralAir'] = joint['CentralAir'].replace({'Y':1, 'N':0})

#Electrical
joint['Electrical'] = joint['Electrical'].fillna(2)
joint['Electrical'] = joint['Electrical'].replace({'Mix':1, 'FuseP':2, 'FuseF':3, 'FuseA':4, 'SBrkr':5})

#kitchenQual
joint['KitchenQual'] = joint['KitchenQual'].replace({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})

#FireplceQu
joint['FireplaceQu'] = joint['FireplaceQu'].fillna(1)
joint['FireplaceQu'] = joint['FireplaceQu'].replace({'Po':2, 'Fa':3, 'TA':4, 'Gd':5, 'Ex':6})

#GarageType
imputed_data_GYB = imputer_mode.fit_transform(joint['GarageType'].values.reshape(2919,1))
joint['GarageType'] = imputed_data_GYB

#GarageYrBlt
imputed_data_GYB = imputer_mode.fit_transform(joint['GarageYrBlt'].values.reshape(2919,1))
joint['GarageYrBlt'] = imputed_data_GYB

#GarageFinish
joint['GarageFinish'] = joint['GarageFinish'].fillna(1)
joint['GarageFinish'] = joint['GarageFinish'].replace({'Unf':2, 'RFn':3, 'Fin':4})

#GarageQual
joint['GarageQual'] = joint['GarageQual'].fillna(1)
joint['GarageQual'] = joint['GarageQual'].replace({'Po':2, 'Fa':3, 'TA':4, 'Gd':5, 'Ex':6})

#GarageCond
joint['GarageCond'] = joint['GarageCond'].fillna(1)
joint['GarageCond'] = joint['GarageCond'].replace({'Po':2, 'Fa':3, 'TA':4, 'Gd':5, 'Ex':6})

#PavedDrive
joint['PavedDrive'] = joint['PavedDrive'].replace({'N':1, 'P':2, 'Y':3})

#PoolQC
joint['PoolQC'] = joint['PoolQC'].fillna(1)
joint['PoolQC'] = joint['PoolQC'].replace({'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})

#Fence
joint['Fence'] = joint['Fence'].fillna('None')

#BsmtFinSF1         1
imputed_data = imputer.fit_transform(joint['BsmtFinSF1'].values.reshape(2919, 1))
joint['BsmtFinSF1'] = imputed_data
# BsmtFinSF2         1
joint['BsmtFinSF2'] = joint['BsmtFinSF2'].fillna(0)
# BsmtFullBath       2
imputed_data = imputer_mode.fit_transform(joint['BsmtFullBath'].values.reshape(2919,1))
joint['BsmtFullBath'] = imputed_data
# BsmtHalfBath       2
imputed_data = imputer_mode.fit_transform(joint['BsmtHalfBath'].values.reshape(2919,1))
joint['BsmtHalfBath'] = imputed_data
# BsmtUnfSF          1
imputed_data = imputer.fit_transform(joint['BsmtUnfSF'].values.reshape(2919, 1))
joint['BsmtUnfSF'] = imputed_data
# GarageArea         1
imputed_data = imputer.fit_transform(joint['GarageArea'].values.reshape(2919, 1))
joint['GarageArea'] = imputed_data
# GarageCars         1
imputed_data = imputer_mode.fit_transform(joint['GarageCars'].values.reshape(2919,1))
joint['GarageCars'] = imputed_data
# KitchenQual        1
imputed_data = imputer_mode.fit_transform(joint['KitchenQual'].values.reshape(2919,1))
joint['KitchenQual'] = imputed_data
# TotalBsmtSF        1
imputed_data = imputer.fit_transform(joint['TotalBsmtSF'].values.reshape(2919, 1))
joint['TotalBsmtSF'] = imputed_data
# Utilities          2
#BsmtQual, ExterQual, Utilities
joint['BsmtQual'] = joint['BsmtQual'].astype(int)
joint['ExterQual'] = joint['ExterQual'].astype(int)
joint['Utilities'] = joint['Utilities'].astype(int)


In [48]:
temp = joint.dtypes[train.dtypes != 'object'].index
skewed_fet = joint[temp]
skewed = skewed_fet.apply(lambda x: skew(x))
# print(skewed)
skewed = skewed[abs(skewed) > 0.75]

from scipy.special import boxcox1p
skewed_fetures = skewed.index
# print(skewed_fetures)
lam=0.17
for feat in skewed_fetures:
    joint[feat] = boxcox1p(joint[feat], lam)

In [121]:
lis_one_hot = ['MSZoning', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2',
                'BldgType', 'HouseStyle', 'RoofMatl','RoofStyle', 'Exterior1st', 'Exterior2nd',
                'MasVnrType', 'Foundation', 'Heating', 'Functional', 'GarageType', 'Fence',
                'MiscFeature', 'SaleType', 'SaleCondition']

one_hot = pd.get_dummies(joint, columns=lis_one_hot)

In [122]:
Ids = one_hot['Id']
train = one_hot.iloc[:1460]
test = one_hot.iloc[1460:]
# id_train = one_hot['Id'].iloc[:1461]
id_test = one_hot['Id'].iloc[1460:]
# prnt(id_test)
# print(temp_train)

In [138]:
def Kfold_cv(kfolds, model, data, target):
    scores = []
    train = data.copy()
    for i,(train_index, test_index) in enumerate(kfolds.split(train)):
        print("---fold--- " ,i)
        pred_list = []
        train_kf = train.iloc[train_index,:]
        test_kf = train.iloc[test_index,:]
        tr_target = target.iloc[train_index]
        va_target = target.iloc[test_index]
        for model_ in model:
            print("model")
            model_.fit(train_kf, tr_target)
            pred_list.append(model_.predict(test_kf))
        stack_error = sqrt(mean_squared_error(va_target, np.mean(pred_list, axis=0)))
        scores.append(stack_error)
    
    return np.mean(scores)

In [124]:
Y = train['SalePrice']
train = train.drop(['Id', 'SalePrice'], axis=1)
train_x, val_x, train_y, val_y = train_test_split(train, Y, random_state = 42)
print(train_x.shape, val_x.shape, train_y.shape, val_y.shape)

(1095, 222) (365, 222) (1095,) (365,)


In [125]:
model_rr = RandomForestRegressor(n_estimators=200, max_depth = 15, max_features=65)
# model.fit(train, Y)


model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

In [140]:
models = [model_rr, model_xgb]

In [42]:
test = test.drop(['SalePrice', 'Id'], axis=1)
submission = model.predict(test)
submission = {'SalePrice':submission}
sub = pd.concat([pd.DataFrame(id_test), pd.DataFrame(submission)], axis=1)
sub.to_csv('~/python/kaggle/neew.csv', index=False)

In [56]:
kaka = RandomForestRegressor(n_estimators=200, max_depth = 15, max_features=65)
kaka.fit(train_x, train_y)
print(sqrt(mean_squared_error(np.log(train_y), np.log(model.predict(train_x)))))
print(sqrt(mean_squared_error(np.log(val_y), np.log(model.predict(val_x)))))

0.0074443003175821455
0.006861518589716554


In [141]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=4, shuffle=True)

score = Kfold_cv(kf, models, train, Y)
print(score)

---fold---  0
model
---fold---  1
model
---fold---  2
model
---fold---  3
model
0.13904078742665987


rr  = 0.13904078742665987
rr,xgb  = 0.12833794453094435
